# Tracking how often mice visit leaf/end nodes

In [8]:
# Imports
from __future__ import print_function
import pickle
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches
from copy import deepcopy
import plotly.graph_objects as go
from scipy.optimize import curve_fit
from dataclasses import make_dataclass
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import sys

module_path = 'code' 
if module_path not in sys.path:
    sys.path.append(module_path)
    
# Markus's code
from MM_Plot_Utils import plot, hist
from MM_Maze_Utils import *
from MM_Traj_Utils import *

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [9]:
# Some lists of nicknames for mice
RewNames=['B1','B2','B3','B4','C1','C3','C6','C7','C8','C9']
UnrewNames=['B5','B6','B7','D3','D4','D5','D6','D7','D8','D9']
AllNames=RewNames+UnrewNames
UnrewNamesSub=['B5','B6','B7','D3','D4','D5','D7','D8','D9'] # excluding D6 which barely entered the maze

In [10]:
# List number of bouts for each animal
print('mouse  bouts')
to=0
max_bouts = 0
for nickname in AllNames:
    tf = LoadTraj(nickname+'-tf')
    print(nickname+'     ',len(tf.no))
    
    if len(tf.no) > max_bouts:
        max_bouts = len(tf.no)
        
    to+=len(tf.no)
print('Total  ',to)

mouse  bouts
B1      104
B2      166
B3      147
B4      238
C1      173
C3      339
C6      150
C7      135
C8      164
C9      127
B5      245
B6      223
B7      204
D3      245
D4      319
D5      180
D6      59
D7      150
D8      109
D9      102
Total   3579


In [ ]:
# define cell numbers of end/leaf nodes
end_nodes = list(range(63,127))

In [ ]:
# Create table with end node visit frequency
'''
visits[nickname][bout][node]: tallies number of times an end node was visited 
                              (a valid visit when the mouse had not already been at the node in the previous frame)
'''
visits = {}
for nickname in AllNames:
    
    # Loading mouse trajectory
    tf = LoadTraj(nickname+'-tf')
    
    # Initializing end node visit dictionary
    visits[nickname] = {}
    
    for bout, node_array in enumerate(tf.no[2:]):
        visits[nickname][bout] = {}
        prior_node = 0
        
        for node in end_nodes:
            visits[nickname][bout][node] = 0
        
        for node,frame in node_array:           
            # checking if the cell visited in the current frame is an end node
            if node in end_nodes and node != prior_node:
                visits[nickname][bout][node] += 1
                
            # keeping track of the mostly recently visited node
            prior_node = node

## Select results for output

In [ ]:
def f(bout):
    
    boutID = bout - 1

    # Initialize new end node visit dictionary for the selected bout
    visits_bout = {}
    for nickname in AllNames:
        visits_bout[nickname] = {}
        for node in end_nodes:
            try:
                visits_bout[nickname][node] = visits[nickname][boutID][node]
            except:
                visits_bout[nickname][node] = np.nan

    # Create a new dataframe
    df = pd.DataFrame(visits_bout)

    # Print the output
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    print('\nEnd nodes visited during bout %i' %(bout))
    
    return df

In [ ]:
# Draw maze with node numbers
ma=NewMaze(6)
PlotMazeNodes(ma,figsize=6)

In [ ]:
set_bout = 24
interact(f, bout=widgets.IntSlider(min=1, max=max_bouts, step=1, value=set_bout));

In [ ]:
def gen_summary():
    # Creating some summary statistics
    '''
    summary[nickname]['None']: % of bouts where NO end nodes were visited
    summary[nickname]['Once']: % of bouts where there was at most ONE visit to any end node
    summary[nickname]['Multiple']: % of bouts where there was at least TWO visits to any end node
    '''

    summary = {}
    summary_bouts = {}
    for nickname in AllNames:
        summary[nickname] = {}
        summary[nickname]['None'] = 0
        summary[nickname]['Once'] = 0
        summary[nickname]['Multiple'] = 0
        summary_bouts[nickname] = {}
        summary_bouts[nickname]['None'] = []
        summary_bouts[nickname]['Once'] = []
        summary_bouts[nickname]['Multiple'] = []
        total_bouts = len(visits[nickname].keys())

        # Calculating % of bouts where NO end nodes were visited
        no_end_nodes = 0
        for bout in visits[nickname].keys():
            total_visits = np.sum(np.asarray(list(visits[nickname][bout].values())))

            if total_visits == 0:
                no_end_nodes += 1
                summary_bouts[nickname]['None'].extend([bout+1])

        summary[nickname]['None'] = np.round(no_end_nodes * 100 / total_bouts)

        # Calculating % of bouts where there was at most only ONE visit to any end node
        one_end_node = 0
        multiple_end_node = 0
        for bout in visits[nickname].keys():
            total_visits = np.sum(np.asarray(list(visits[nickname][bout].values())))

            if total_visits != 0:
                one_visit = True
                for node in visits[nickname][bout]:

                    # Checking if an end node has been visited multiple times
                    if visits[nickname][bout][node] > 1:
                        one_visit = False

                if one_visit:
                    one_end_node += 1
                    summary_bouts[nickname]['Once'].extend([bout+1])
                elif not one_visit:
                    multiple_end_node += 1
                    summary_bouts[nickname]['Multiple'].extend([bout+1])

                summary[nickname]['Once'] = np.round(one_end_node * 100 / total_bouts)
                summary[nickname]['Multiple'] = np.round(multiple_end_node * 100 / total_bouts)
                
    return summary, summary_bouts

In [ ]:
summary, summary_bouts = gen_summary()
df_summary = pd.DataFrame(summary)
print("\n None: % of bouts where NO end nodes were visited" 
       "\n Once: % of bouts where any end node was visited ONCE at most"
       "\n Multiple: % of bouts where any end node was visited MORE than ONCE")
df_summary

In [ ]:
# Example use of summary_bouts: Checking which bouts for 'B1' involved no visits to end nodes
summary_bouts['B1']['Once']